In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["OPENAI_MODEL_NAME"] = "gpt-4.1-nano"

In [3]:
from crewai import Crew, Agent, Task
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, YoutubeChannelSearchTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
youtube_tool = YoutubeChannelSearchTool()

researcher = Agent(
    role="Senior Researcher",
    goal="Search the web, extract and analyze information.",
    backstory="""
    You produce the highest quality research possible.
    You use multiple sources of information and you always double check your sources to make sure they are true and up to date.
    You want to impress your coworkers with your work.
    """,
    allow_delegation=False,
    tools=[
        search_tool,
        scrape_tool,
        youtube_tool,
    ],
    max_iter=3,
    verbose=True,
)

marketer = Agent(
    role="Senior Marketer",
    goal="Come up with ideas that generate viral and useful content.",
    backstory="""
    You work at a marketing agency.
    You are the best at coming up with ideas to make content go viral.
    Your ideas are used for video, advertising, social media marketing, the content you produce appeals to a young audience.
    """,
    verbose=True,
)

writer = Agent(
    role="Senior Writer",
    goal="Write scripts for viral Youtube videos.",
    backstory="""
    You write scripts for videos that keep people engaged and entertained.
    Your content is easy and fun to watch, it is informative and it makes people want to share it with their friends.
    You are working for a very important client.
    """,
    verbose=True,
)

/Users/suajeong/code/gpt-assistants-fullstack/env/lib/python3.11/site-packages/alembic/config.py:564: DeprecationWarning: No path_separator found in configuration; falling back to legacy splitting on spaces, commas, and colons for prepend_sys_path.  Consider adding path_separator=os to Alembic config.
  util.warn_deprecated(


In [6]:
brainstorm_task = Task(
    description="Come up with 5 video ideas for a Youtube channel in the {industry} industry",
    agent=marketer,
    expected_output="Your answer MUST be a list of 5 ideas for a Youtube video with an explanation of what the angle of the video would be.",
    output_file="ideas_task.md",
    human_input=True,
)

selection_task = Task(
    description="Select a video idea that has the highest potential of going viral.",
    agent=writer,
    expected_output="Your answer MUST include the idea that was selected as well as an explanation of why that selection was made.",
    human_input=True,
    context=[brainstorm_task],
    output_file="selection_task.md",
)

research_task = Task(
    description="Do all the research required to write the script of a medium length video about the selected idea.",
    agent=researcher,
    expected_output="You answer must have all the information a writer would need to write a Youtube script.",
    async_execution=True,
    context=[
        selection_task,
    ],
    output_file="research_task.md",
)

competitors_task = Task(
    description="Search for videos or articles in the {industry} industry that are similar to the video idea we are working on and suggest ways our video can be different from theirs.",
    expected_output="Your answer must have a list of suggestions writers can follow to make sure the video is as unique and as different from competitors as possible.",
    agent=researcher,
    async_execution=True,
    context=[
        selection_task,
    ],
    output_file="competitors_task.md",
)

inspiration_task = Task(
    description="Search for videos or articles that are similar to the video idea we are working on but from other industries.",
    expected_output="Your answer must have a list of examples of articles and videos that have a similar angle as the video we are making but that are in different industries.",
    agent=researcher,
    async_execution=True,
    context=[
        selection_task,
    ],
    output_file="inspiration_task.md",
)

# context uses context window like gpt-4o, gpt-4 turbo
script_task = Task(
    description="Write the script for a Youtube video for a channel in the {industry} industry.",
    expected_output="A script for a Youtube video with a title, an introduction, at least three sections, and an outro. Make sure to also include the prompt to generate a thumbnail for the video.",
    agent=writer,
    context=[
        selection_task,
        research_task,
        competitors_task,
        inspiration_task,
    ],
    output_file="script_task.md",
)

In [7]:
crew = Crew(
    agents=[
        researcher,
        marketer,
        writer,
    ],
    tasks=[
        brainstorm_task,
        selection_task,
        research_task,
        competitors_task,
        inspiration_task,
        script_task,
    ],
    memory=True,
    planning=True,
)

/Users/suajeong/code/gpt-assistants-fullstack/env/lib/python3.11/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


In [8]:
result = crew.kickoff(
    inputs=dict(
        industry="Perfume",
    ),
)

# Agent: Senior Marketer
## Task: Come up with 5 video ideas for a Youtube channel in the Perfume industry1. Begin by brainstorming topics relevant to the perfume industry that could attract a wide audience. 2. Consider current trends in fragrances, consumer preferences, and popular demand. 3. Create a list of 5 distinct video ideas, ensuring each idea has a unique angle. For example: 'Top 10 Summer Fragrances for 2023,' 'Behind the Scenes: How Perfumes are Made,' 'Exploring the Science of Scents,' 'Fragrance Layering: Create Your Signature Scent,' 'History of Iconic Perfumes.' 4. Provide a brief explanation of each idea focusing on potential audience appeal, educational value, and entertainment aspects to help generate interest and insights.


# Agent: Senior Marketer
## Final Answer: 
1. "Top 10 Summer Fragrances for 2024: Must-Have Scents for the Hot Season" – This video highlights the most popular and trending summer perfumes, appealing to viewers eager to update their scent wardro

/Users/suajeong/code/gpt-assistants-fullstack/env/lib/python3.11/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
/Users/suajeong/code/gpt-assistants-fullstack/env/lib/python3.11/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


# Agent: Senior Writer
## Task: Select a video idea that has the highest potential of going viral.1. Review the video ideas generated in Task 1, considering each one's uniqueness, relevance, and viral potential. 2. Analyze current trends in the perfume market and social media to identify what content resonates with the target audience. 3. Select the video idea that could captivate viewers the most. For example, choosing 'Top 10 Summer Fragrances for 2023' due to its timely nature and relevance. 4. Explain the criteria that led to this choice, such as the ability to provide value, engage viewers, and create shareable content.


# Agent: Senior Writer
## Final Answer: 
The video idea with the highest potential of going viral is **"Top 10 Summer Fragrances for 2024: Must-Have Scents for the Hot Season."** 

This choice was made based on several compelling factors. Firstly, it taps into current trends, as consumers are always eager to discover new and trending products—especially seasonal 

/Users/suajeong/code/gpt-assistants-fullstack/env/lib/python3.11/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
/Users/suajeong/code/gpt-assistants-fullstack/env/lib/python3.11/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


# Agent: Senior Researcher
## Task: Do all the research required to write the script of a medium length video about the selected idea.1. Utilize the 'Search the internet with Serper' tool to research the chosen video topic. For example, search for 'Top Summer Fragrances 2023.' 2. Collect data on summer perfumes including top brands, fragrance notes, user reviews, and trends observed in fragrance choices for summer. 3. Aggregate information from multiple sources to get a comprehensive view. 4. Identify key points that should be included in the script such as perfume notes, tips for choosing a summer fragrance, and potential audience engagement strategies.


# Agent: Senior Researcher
## Final Answer: 
Search the internet with Serper  
search_query: {"description": "Top Summer Fragrances 2024", "type": "search"}


# Agent: Senior Writer
## Task: Write the script for a Youtube video for a channel in the Perfume industry.1. Begin drafting the script based on the research gathered from Task